# 📘 Notebook 2: Deep Learning on GPU (SPCS) — CNN vs DNN

## Introduction

This notebook compares **CNN (Convolutional Neural Network)** vs **DNN (Deep Neural Network)** for wafer yield prediction using GPU compute on SPCS.

| Model | Architecture | Strengths |
|-------|--------------|-----------|
| **DNN** | Multi-Layer Perceptron | Universal approximator, good baseline |
| **CNN** | 1D Convolutional Network | Captures local patterns in sequential features |

### What's Covered

| Section | Topic |
|---------|-------|
| 1 | Load ML Dataset via Data Connector → PyTorch |
| 2 | GPU Setup |
| 3 | Define DNN & CNN Architectures |
| 4 | Train Both Models |
| 5 | Log to Snowflake ML Experiments |
| 6 | Register Champion Model to Registry |

### References

- [Snowflake ML Data Connector](https://docs.snowflake.com/en/developer-guide/snowflake-ml/load-data)
- [Snowflake ML Experiments](https://docs.snowflake.com/en/developer-guide/snowflake-ml/experiments)
- [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/overview)


In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================

import numpy as np
import pandas as pd
from datetime import datetime
from typing import Dict, List

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Snowflake
from snowflake.snowpark.context import get_active_session
from snowflake.ml import dataset
from snowflake.ml.data import DataConnector
from snowflake.ml.experiment import ExperimentTracking
from snowflake.ml.registry import Registry

print("✅ Imports complete")

In [ ]:
# ============================================================================
# SESSION & EXPERIMENT SETUP
# ============================================================================

session = get_active_session()
session.sql("USE DATABASE WAFER_YIELD_DEMO").collect()
session.sql("USE SCHEMA RAW_DATA").collect()  # Same schema where Dataset was created in Notebook 1

# Initialize Snowflake ML Experiments
exp = ExperimentTracking(session=session)
EXPERIMENT_NAME = "WAFER_YIELD_CNN_VS_DNN"
exp.set_experiment(EXPERIMENT_NAME)

print(f"✅ Session: {session.get_current_database()}.{session.get_current_schema()}")
print(f"✅ Experiment: {EXPERIMENT_NAME}")


---
## Section 1 — Load ML Dataset via Data Connector

### Why Use the Snowflake Data Connector?

The **DataConnector** is Snowflake's optimized API for loading data into ML workflows. Key benefits:

| Benefit | Description |
|---------|-------------|
| **Parallel Reads** | Distributes data loading across multiple compute nodes using Ray |
| **Direct Integration** | Works seamlessly with Snowflake Datasets and Snowpark DataFrames |
| **Framework Support** | Converts directly to pandas, PyTorch, or TensorFlow datasets |
| **Streaming** | Data loaded in streaming fashion for memory efficiency |

### Data Loading Options

```
DataConnector.from_dataset(snowflake_dataset)  → Load from ML Dataset
DataConnector.from_dataframe(snowpark_df)      → Load from Snowpark DataFrame
```

### Conversion Methods

```python
data_connector.to_pandas()           # → pandas DataFrame
data_connector.to_torch_dataset()    # → PyTorch Dataset  
data_connector.to_tf_dataset()       # → TensorFlow Dataset
```

**Reference**: [Snowflake ML Load Data Documentation](https://docs.snowflake.com/en/developer-guide/snowflake-ml/load-data)

In [ ]:
# ============================================================================
# LOAD ML DATASET → DATA CONNECTOR → PYTORCH
# ============================================================================

DATASET_NAME = "WAFER_YIELD_TRAINING_DATASET"
DATASET_VERSION = "v1"
BATCH_SIZE = 256

# Load ML Dataset (created in Notebook 01)
wafer_dataset = dataset.load_dataset(session, DATASET_NAME, DATASET_VERSION)

# Create DataConnector and convert directly to PyTorch dataset
data_connector = DataConnector.from_dataset(wafer_dataset)
torch_dataset = data_connector.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=True)

# Create DataLoader (batch_size=None because batching handled by to_torch_dataset)
dataloader = DataLoader(torch_dataset, batch_size=None)

# Define columns
label_col = "YIELD_GOOD"
EXCLUDE_COLS = ['WAFER_ID', 'YIELD_GOOD', 'YIELD_SCORE', 'DOMINANT_DEFECT_TYPE']

print(f"✅ ML Dataset loaded: {DATASET_NAME} v{DATASET_VERSION}")
print(f"✅ PyTorch DataLoader ready (batch_size={BATCH_SIZE})")


In [ ]:
# ============================================================================
# INSPECT FIRST BATCH & DETERMINE FEATURE COLUMNS
# ============================================================================

# Get first batch to discover columns dynamically
first_batch = next(iter(dataloader))
all_columns = list(first_batch.keys())
feature_cols = [c for c in all_columns if c not in EXCLUDE_COLS]
input_dim = len(feature_cols)

print(f"📊 Batch Structure:")
print(f"   All columns: {len(all_columns)}")
print(f"   Feature columns: {input_dim}")
print(f"   Label column: {label_col}")
print(f"\n   Features: {feature_cols[:5]}...")

---
## Section 2 — GPU Setup

In [ ]:
# ============================================================================
# GPU CONFIGURATION
# ============================================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


---
## Section 3 — Define DNN & CNN Architectures


In [ ]:
# ============================================================================
# DNN (DEEP NEURAL NETWORK) ARCHITECTURE
# ============================================================================

class WaferYieldDNN(nn.Module):
    """Multi-layer perceptron for wafer yield classification."""
    
    def __init__(self, input_dim, hidden_dims=[256, 128, 64, 32], dropout=0.3):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h),
                nn.BatchNorm1d(h),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h
        layers.extend([nn.Linear(prev_dim, 1), nn.Sigmoid()])
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

dnn_model = WaferYieldDNN(input_dim).to(device)
print(f"🧠 DNN: {sum(p.numel() for p in dnn_model.parameters()):,} parameters")


In [ ]:
# ============================================================================
# CNN (1D CONVOLUTIONAL NEURAL NETWORK) ARCHITECTURE
# ============================================================================

class WaferYieldCNN(nn.Module):
    """1D CNN that treats features as a sequence."""
    
    def __init__(self, input_dim, channels=[64, 128, 256], kernel_size=3, fc_dim=128, dropout=0.3):
        super().__init__()
        conv_layers = []
        in_ch = 1
        for out_ch in channels:
            conv_layers.extend([
                nn.Conv1d(in_ch, out_ch, kernel_size, padding=kernel_size//2),
                nn.BatchNorm1d(out_ch),
                nn.ReLU(),
                nn.MaxPool1d(2)
            ])
            in_ch = out_ch
        self.conv = nn.Sequential(*conv_layers)
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(channels[-1], fc_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(fc_dim, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = x.unsqueeze(1)  # (batch, 1, features)
        return self.fc(self.conv(x))

cnn_model = WaferYieldCNN(input_dim).to(device)
print(f"🧠 CNN: {sum(p.numel() for p in cnn_model.parameters()):,} parameters")


---
## Section 4 — Train Both Models


In [ ]:
# ============================================================================
# TRAINING CONFIGURATION
# ============================================================================

EPOCHS = 50
LR = 0.001
criterion = nn.BCELoss()

# Store metrics for comparison
dnn_metrics = {}
cnn_metrics = {}

print(f"⚙️ Training Config: epochs={EPOCHS}, lr={LR}, batch_size={BATCH_SIZE}")


In [ ]:
# ============================================================================
# TRAIN DNN & LOG TO EXPERIMENTS
# ============================================================================

run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

with exp.start_run(run_name=f"DNN_{run_timestamp}") as dnn_run:
    # Log hyperparameters
    exp.log_params({
        "model_type": "DNN",
        "hidden_dims": "[256, 128, 64, 32]",
        "dropout": 0.3,
        "epochs": EPOCHS,
        "learning_rate": LR,
        "batch_size": BATCH_SIZE
    })
    
    print("🚀 Training DNN...")
    dnn_start = datetime.now()
    optimizer = optim.AdamW(dnn_model.parameters(), lr=LR)
    
    correct, total = 0, 0
    for epoch in range(EPOCHS):
        dnn_model.train()
        epoch_loss = 0
        batch_count = 0
        
        # Reload for each epoch (streaming dataset)
        torch_dataset = data_connector.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=True)
        dataloader = DataLoader(torch_dataset, batch_size=None)
        
        for batch in dataloader:
            y = batch.pop(label_col).float().squeeze().unsqueeze(1).to(device)
            X = torch.stack([batch[col].float().squeeze() for col in feature_cols], dim=1).to(device)
            
            optimizer.zero_grad()
            pred = dnn_model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            batch_count += 1
            
            if epoch == EPOCHS - 1:
                correct += ((pred >= 0.5).float() == y).sum().item()
                total += y.size(0)
        
        if (epoch + 1) % 10 == 0:
            print(f"   Epoch {epoch+1}: loss={epoch_loss/batch_count:.4f}")
    
    dnn_time = (datetime.now() - dnn_start).total_seconds()
    dnn_accuracy = correct / total if total > 0 else 0
    dnn_loss = epoch_loss / batch_count
    
    # Log final metrics to experiment tracking
    exp.log_metrics({
        "accuracy": dnn_accuracy,
        "final_loss": dnn_loss,
        "training_time_seconds": dnn_time
    })
    
    print(f"✅ DNN logged: accuracy={dnn_accuracy:.4f}, time={dnn_time:.1f}s")

In [ ]:
# ============================================================================
# TRAIN CNN & LOG TO EXPERIMENTS
# ============================================================================

with exp.start_run(run_name=f"CNN_{run_timestamp}") as cnn_run:
    # Log hyperparameters
    exp.log_params({
        "model_type": "CNN",
        "channels": "[64, 128, 256]",
        "kernel_size": 3,
        "dropout": 0.3,
        "epochs": EPOCHS,
        "learning_rate": LR,
        "batch_size": BATCH_SIZE
    })
    
    print("🚀 Training CNN...")
    cnn_start = datetime.now()
    optimizer = optim.AdamW(cnn_model.parameters(), lr=LR)
    
    correct, total = 0, 0
    for epoch in range(EPOCHS):
        cnn_model.train()
        epoch_loss = 0
        batch_count = 0
        
        # Reload for each epoch (streaming dataset)
        torch_dataset = data_connector.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=True)
        dataloader = DataLoader(torch_dataset, batch_size=None)
        
        for batch in dataloader:
            y = batch.pop(label_col).float().squeeze().unsqueeze(1).to(device)
            X = torch.stack([batch[col].float().squeeze() for col in feature_cols], dim=1).to(device)
            
            optimizer.zero_grad()
            pred = cnn_model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            batch_count += 1
            
            if epoch == EPOCHS - 1:
                correct += ((pred >= 0.5).float() == y).sum().item()
                total += y.size(0)
        
        if (epoch + 1) % 10 == 0:
            print(f"   Epoch {epoch+1}: loss={epoch_loss/batch_count:.4f}")
    
    cnn_time = (datetime.now() - cnn_start).total_seconds()
    cnn_accuracy = correct / total if total > 0 else 0
    cnn_loss = epoch_loss / batch_count
    
    # Log final metrics to experiment tracking
    exp.log_metrics({
        "accuracy": cnn_accuracy,
        "final_loss": cnn_loss,
        "training_time_seconds": cnn_time
    })
    
    print(f"✅ CNN logged: accuracy={cnn_accuracy:.4f}, time={cnn_time:.1f}s")


---
## Section 4b — Distributed Hyperparameter Tuning (DNN)

Use **Snowflake ML Tune** for distributed hyperparameter search across multiple trials.

### Key Concepts

| Component | Description |
|-----------|-------------|
| **TuneConfig** | Search space definition (params to tune) |
| **ScalingConfig** | Compute resources per trial |
| **RandomSearch** | Search algorithm (random sampling) |

**Reference**: [Snowflake ML Hyperparameter Tuning](https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling/tune-model-hyperparameters)


In the example below, these are the explicit differences in these trainings vs the ones above. 

|Aspect	| Baseline	| Tuning |
|------ | ---------| ------- | 
|Architecture	| Fixed 4 layers	|Dynamic 2 layers |
|Epochs	| 50	| 10 (faster trials) |
|Hyperparameters	| Hardcoded	| From search space |
|Runs	| 1	| 8 parallel trials |
|Purpose	| Full training	| Find best config |

The tuner runs train_dnn_trial() 8 times with different configs, then you'd typically retrain the best config with full epochs (50) for the final model.


In [ ]:
# ============================================================================
# DEFINE TRAINABLE FUNCTION FOR TUNING
# ============================================================================

def train_dnn_trial(config: dict) -> dict:
    """
    Training function for a single hyperparameter trial.
    Called by Snowflake ML Tune with different config values.
    """
    # Extract hyperparameters from config
    hidden_dim_1 = config["hidden_dim_1"]
    hidden_dim_2 = config["hidden_dim_2"]
    dropout = config["dropout"]
    lr = config["learning_rate"]
    
    # Build model with trial hyperparameters
    model = nn.Sequential(
        nn.Linear(input_dim, hidden_dim_1),
        nn.BatchNorm1d(hidden_dim_1),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_dim_1, hidden_dim_2),
        nn.BatchNorm1d(hidden_dim_2),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_dim_2, 1),
        nn.Sigmoid()
    ).to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    
    # Train for fewer epochs during tuning
    TUNE_EPOCHS = 10
    
    for epoch in range(TUNE_EPOCHS):
        model.train()
        torch_ds = data_connector.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=True)
        dl = DataLoader(torch_ds, batch_size=None)
        
        for batch in dl:
            y = batch.pop(label_col).float().squeeze().unsqueeze(1).to(device)
            X = torch.stack([batch[col].float().squeeze() for col in feature_cols], dim=1).to(device)
            
            optimizer.zero_grad()
            pred = model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
    
    # Evaluate
    model.eval()
    correct, total = 0, 0
    torch_ds = data_connector.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=False)
    dl = DataLoader(torch_ds, batch_size=None)
    
    with torch.no_grad():
        for batch in dl:
            y = batch.pop(label_col).float().squeeze().unsqueeze(1).to(device)
            X = torch.stack([batch[col].float().squeeze() for col in feature_cols], dim=1).to(device)
            pred = model(X)
            correct += ((pred >= 0.5).float() == y).sum().item()
            total += y.size(0)
    
    accuracy = correct / total if total > 0 else 0
    
    return {"accuracy": accuracy}

print("✅ Training function defined for tuning")


In [ ]:
# ============================================================================
# DEFINE TRAINABLE FUNCTION FOR TUNING
# ============================================================================

def train_dnn_trial():
    """
    Training function for Snowflake ML Tune.
    Uses get_tuner_context() to access hyperparameters and datasets.
    """
    # Get tuner context
    tuner_context = get_tuner_context()
    config = tuner_context.get_hyper_params()
    dataset_map = tuner_context.get_dataset_map()
    
    # Extract hyperparameters
    hidden_dim_1 = int(config["hidden_dim_1"])
    hidden_dim_2 = int(config["hidden_dim_2"])
    dropout = config["dropout"]
    lr = config["learning_rate"]
    
    # Load data from dataset_map
    train_dc = dataset_map["train"]
    
    # Build model with trial hyperparameters
    model = nn.Sequential(
        nn.Linear(input_dim, hidden_dim_1),
        nn.BatchNorm1d(hidden_dim_1),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_dim_1, hidden_dim_2),
        nn.BatchNorm1d(hidden_dim_2),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_dim_2, 1),
        nn.Sigmoid()
    ).to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    
    # Train for fewer epochs during tuning
    TUNE_EPOCHS = 10
    
    for epoch in range(TUNE_EPOCHS):
        model.train()
        torch_ds = train_dc.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=True)
        dl = DataLoader(torch_ds, batch_size=None)
        
        for batch in dl:
            y = batch.pop(label_col).float().squeeze().unsqueeze(1).to(device)
            X = torch.stack([batch[col].float().squeeze() for col in feature_cols], dim=1).to(device)
            
            optimizer.zero_grad()
            pred = model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
    
    # Evaluate
    model.eval()
    correct, total = 0, 0
    torch_ds = train_dc.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=False)
    dl = DataLoader(torch_ds, batch_size=None)
    
    with torch.no_grad():
        for batch in dl:
            y = batch.pop(label_col).float().squeeze().unsqueeze(1).to(device)
            X = torch.stack([batch[col].float().squeeze() for col in feature_cols], dim=1).to(device)
            pred = model(X)
            correct += ((pred >= 0.5).float() == y).sum().item()
            total += y.size(0)
    
    accuracy = correct / total if total > 0 else 0
    
    # Report metrics and model back to tuner
    tuner_context.report(metrics={"accuracy": accuracy}, model=model)

print("✅ Training function defined for Snowflake ML Tune")


In [ ]:
# ============================================================================
# RUN DISTRIBUTED HYPERPARAMETER TUNING (Snowflake ML Tune)
# ============================================================================

# Define search space using Snowflake ML sampling functions
# Can also use lists for grid search: [128, 256, 512]
from snowflake.ml.modeling.tune import Uniform, LogUniform
# Import tuning components (in case not already imported)
from snowflake.ml.modeling.tune import Tuner, TunerConfig, get_tuner_context, Uniform, LogUniform
from snowflake.ml.modeling.tune.search import RandomSearch


search_space = {
    "hidden_dim_1": Uniform(128, 512),
    "hidden_dim_2": Uniform(64, 256),
    "dropout": Uniform(0.2, 0.5),
    "learning_rate": LogUniform(1e-4, 1e-2)
}

# Configure tuning with GPU resources
tuner_config = TunerConfig(
    metric="accuracy",
    mode="max",
    search_alg=RandomSearch(random_state=42),
    num_trials=8,
    resource_per_trial={"GPU": 1, "CPU": 2}  # Request GPU for each trial
)

print("🔧 Starting distributed hyperparameter tuning...")
print(f"   Search space: hidden_dim_1, hidden_dim_2, dropout, learning_rate")
print(f"   Num trials: {tuner_config.num_trials}")
print(f"   Search algorithm: RandomSearch")

# Create dataset_map for tuner
dataset_map = {"train": data_connector}

# Create and run tuner
tuner = Tuner(
    train_func=train_dnn_trial,
    search_space=search_space,
    tuner_config=tuner_config
)

tune_results = tuner.run(dataset_map=dataset_map)

# Get best result
best_config = tune_results.best_result
best_accuracy = tune_results.best_result["accuracy"].iloc[0] if hasattr(tune_results.best_result["accuracy"], "iloc") else tune_results.best_result["accuracy"]

print(f"\n🏆 Best Trial Results:")
print(f"   Accuracy: {best_accuracy:.4f}")
print(f"   Config: {best_config}")


In [ ]:
# ============================================================================
# RETRAIN BEST MODEL WITH FULL EPOCHS
# ============================================================================

print("🚀 Retraining with best hyperparameters for full epochs...")

# Debug: print the structure of best_result
print(f"   Best result type: {type(tune_results.best_result)}")
print(f"   Best result: {tune_results.best_result}")

# Extract best hyperparameters - handle different result structures
try:
    # If it's a DataFrame row
    if hasattr(tune_results.best_result, "iloc"):
        row = tune_results.best_result.iloc[0] if len(tune_results.best_result) > 0 else tune_results.best_result
        best_hidden_1 = int(row["hidden_dim_1"])
        best_hidden_2 = int(row["hidden_dim_2"])
        best_dropout = float(row["dropout"])
        best_lr = float(row["learning_rate"])
    # If it's a dict
    elif isinstance(tune_results.best_result, dict):
        best_hidden_1 = int(tune_results.best_result["hidden_dim_1"])
        best_hidden_2 = int(tune_results.best_result["hidden_dim_2"])
        best_dropout = float(tune_results.best_result["dropout"])
        best_lr = float(tune_results.best_result["learning_rate"])
    else:
        # Try accessing as attributes or columns
        best_hidden_1 = int(tune_results.best_result.hidden_dim_1)
        best_hidden_2 = int(tune_results.best_result.hidden_dim_2)
        best_dropout = float(tune_results.best_result.dropout)
        best_lr = float(tune_results.best_result.learning_rate)
except Exception as e:
    print(f"   Error extracting params: {e}")
    print(f"   Using default values from baseline")
    best_hidden_1 = 256
    best_hidden_2 = 128
    best_dropout = 0.3
    best_lr = 0.001

print(f"   Best config: hidden_dims=[{best_hidden_1}, {best_hidden_2}], dropout={best_dropout:.2f}, lr={best_lr:.6f}")

# Build tuned model
tuned_model = nn.Sequential(
    nn.Linear(input_dim, best_hidden_1),
    nn.BatchNorm1d(best_hidden_1),
    nn.ReLU(),
    nn.Dropout(best_dropout),
    nn.Linear(best_hidden_1, best_hidden_2),
    nn.BatchNorm1d(best_hidden_2),
    nn.ReLU(),
    nn.Dropout(best_dropout),
    nn.Linear(best_hidden_2, 1),
    nn.Sigmoid()
).to(device)

# Train with full epochs
tuned_start = datetime.now()
optimizer = optim.AdamW(tuned_model.parameters(), lr=best_lr)

correct, total = 0, 0
for epoch in range(EPOCHS):  # Full 50 epochs
    tuned_model.train()
    epoch_loss = 0
    batch_count = 0
    
    torch_dataset = data_connector.to_torch_dataset(batch_size=BATCH_SIZE, shuffle=True)
    dataloader = DataLoader(torch_dataset, batch_size=None)
    
    for batch in dataloader:
        y = batch.pop(label_col).float().squeeze().unsqueeze(1).to(device)
        X = torch.stack([batch[col].float().squeeze() for col in feature_cols], dim=1).to(device)
        
        optimizer.zero_grad()
        pred = tuned_model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        batch_count += 1
        
        if epoch == EPOCHS - 1:
            correct += ((pred >= 0.5).float() == y).sum().item()
            total += y.size(0)
    
    if (epoch + 1) % 10 == 0:
        print(f"   Epoch {epoch+1}: loss={epoch_loss/batch_count:.4f}")

tuned_time = (datetime.now() - tuned_start).total_seconds()
tuned_accuracy = correct / total if total > 0 else 0
tuned_loss = epoch_loss / batch_count

print(f"✅ Tuned DNN: accuracy={tuned_accuracy:.4f}, time={tuned_time:.1f}s")


In [ ]:
# ============================================================================
# LOG TUNED MODEL TO EXPERIMENTS
# ============================================================================

with exp.start_run(run_name=f"DNN_tuned_{run_timestamp}") as tuned_run:
    # Log hyperparameters (from tuning)
    exp.log_params({
        "model_type": "DNN_tuned",
        "hidden_dim_1": best_hidden_1,
        "hidden_dim_2": best_hidden_2,
        "dropout": best_dropout,
        "learning_rate": best_lr,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "tuning_trials": tuner_config.num_trials
    })
    
    # Log final metrics (from full training)
    exp.log_metrics({
        "accuracy": tuned_accuracy,
        "final_loss": tuned_loss,
        "training_time_seconds": tuned_time
    })

print(f"✅ Tuned DNN logged to experiments: DNN_tuned_{run_timestamp}")
print(f"   Accuracy: {tuned_accuracy:.4f} (vs baseline DNN: {dnn_accuracy:.4f})")

In [ ]:
# ============================================================================
# COMPARE ALL 3 MODELS
# ============================================================================

print("=" * 70)
print("📊 EXPERIMENT COMPARISON - All Models")
print("=" * 70)
print(f"{'Model':<15} {'Accuracy':<12} {'Loss':<12} {'Time (s)':<10} {'Params':<12}")
print("-" * 70)
print(f"{'DNN (baseline)':<15} {dnn_accuracy:<12.4f} {dnn_loss:<12.4f} {dnn_time:<10.1f} {sum(p.numel() for p in dnn_model.parameters()):,}")
print(f"{'CNN (baseline)':<15} {cnn_accuracy:<12.4f} {cnn_loss:<12.4f} {cnn_time:<10.1f} {sum(p.numel() for p in cnn_model.parameters()):,}")
print(f"{'DNN (tuned)':<15} {tuned_accuracy:<12.4f} {tuned_loss:<12.4f} {tuned_time:<10.1f} {sum(p.numel() for p in tuned_model.parameters()):,}")
print("=" * 70)

# Determine winner from all 3 models
all_models = [
    ("DNN_baseline", dnn_model, dnn_accuracy, dnn_loss),
    ("CNN_baseline", cnn_model, cnn_accuracy, cnn_loss),
    ("DNN_tuned", tuned_model, tuned_accuracy, tuned_loss)
]

# Sort by accuracy (descending)
all_models_sorted = sorted(all_models, key=lambda x: x[2], reverse=True)

print("\n🏆 RANKING:")
for i, (name, model, acc, loss) in enumerate(all_models_sorted, 1):
    medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
    print(f"   {medal} #{i}: {name} (accuracy={acc:.4f}, loss={loss:.4f})")

# Set winner for model registration
winner_name, winner_model, winner_accuracy, winner_loss = all_models_sorted[0]

print(f"\n📊 View all runs in Snowsight:")
print(f"   Database > WAFER_YIELD_DEMO > ML > Experiments > {EXPERIMENT_NAME}")

---
## Section 6 — Register Champion Model to Model Registry

### What is Snowflake Model Registry?

The **Snowflake Model Registry** is a centralized repository for managing ML models throughout their lifecycle:

| Capability | Description |
|------------|-------------|
| **Model Versioning** | Track multiple versions of the same model |
| **Metadata Storage** | Store metrics, parameters, and artifacts |
| **Model Staging** | Manage model lifecycle (dev → staging → production) |
| **Access Control** | RBAC for model access and deployment |
| **Lineage Tracking** | Track which data/features were used to train |

### The `registry.log_model()` Process

```python
registry.log_model(
    model=winner_model,           # PyTorch model object
    model_name="WAFER_YIELD_...", # Unique identifier
    version_name="v1",            # Version tag
    comment="Description...",     # Human-readable notes
    metrics={...},                # Performance metrics
    sample_input_data={...}       # For signature inference
)
```

### What Gets Stored?

| Component | Description |
|-----------|-------------|
| **Model Artifact** | Serialized model weights (PyTorch state_dict) |
| **Model Signature** | Input/output schema inferred from sample_input_data |
| **Metrics** | accuracy, loss, or any custom metrics |
| **Metadata** | comment, creation time, user, version |
| **Dependencies** | Python packages required for inference |

### Model Lifecycle Stages

```
Development → Staging → Production → Archived
     │           │           │
     └───────────┴───────────┘
        Promote via set_alias() or tags
```

### Best Practices

1. **Semantic Versioning**: Use `v1`, `v2`, or `v1.0.1` for versions
2. **Include Metrics**: Always log accuracy, loss for comparison
3. **Sample Input**: Provide sample data for signature inference
4. **Descriptive Comments**: Document what changed in each version

**Reference**: [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/overview)

In [ ]:
# ============================================================================
# REGISTER CHAMPION MODEL TO SNOWFLAKE MODEL REGISTRY
# ============================================================================
import pandas as pd

registry = Registry(session=session)

# Move model to CPU for registry (required for signature validation)
winner_model_cpu = winner_model.cpu()
winner_model_cpu.eval()

# Create sample input as pandas DataFrame for signature inference (float32 to match model)
import numpy as np
sample_input_df = pd.DataFrame({col: np.array([0.0], dtype=np.float32) for col in feature_cols})

# Register the winning model (best from all 3: DNN baseline, CNN baseline, DNN tuned)
model_name = f"WAFER_YIELD_{winner_name.upper().replace(' ', '_')}"
model_version = registry.log_model(
    model=winner_model_cpu,
    model_name=model_name,
    version_name="v1",
    comment=f"Champion model ({winner_name}) - best from DNN/CNN/Tuned comparison",
    metrics={
        "accuracy": float(winner_accuracy),
        "final_loss": float(winner_loss),
        "total_params": int(sum(p.numel() for p in winner_model_cpu.parameters()))
    },
    sample_input_data=sample_input_df
)

print(f"✅ Registered Champion Model: {model_name} v1")
print(f"   Model: {winner_name}")
print(f"   Accuracy: {winner_accuracy:.4f}")
print(f"   Loss: {winner_loss:.4f}")
print(f"   Parameters: {sum(p.numel() for p in winner_model.parameters()):,}")
print(f"\n📦 View in Snowsight:")
print(f"   Database > {session.get_current_database()} > ML > Model Registry > {model_name}")
